# Hyperparameter Tuning Using Optuna

## Creating a Class for Easier Hyperparameter Tuning With Optuna

In [6]:
# package imports for OptunaTuner class
import logging
import optuna
import pandas as pd
from typing import Callable
from pickle import dump
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [7]:
class OptunaTuner:
    def __init__(
        self,
        objective: Callable[[optuna.trial.Trial], float],
        pruner: optuna.pruners,
        n_trials: int = 100,
        seed: int = 123,
        study_timeout: int = 5 * 3600,
        direction: str = "minimize",
        study_name: str = None,
        save_plots: bool = False,
    ):
        """Set up the Optuna tuner.

        Args:
            objective (Callable[[optuna.trial.Trial], float]): Objective function to optimize.
            pruner (optuna.pruners): Pruner to use.
            n_trials (int, optional): Number of trials. Defaults to 100.
            seed (int, optional): Random seed to use. Defaults to 123.
            study_timeout (int, optional): Automatic termination after how many seconds. Defaults to 5*3600.
            direction (str, optional): Whether to minimize or maximize objective. Defaults to "minimize".
            study_name (str, optional): Name of the optuna study. Defaults to None.
            save_plots (bool, optional): Whether to save the plotly plots or not. Defaults to False.
        """
        self.objective = objective
        self.pruner = pruner
        self.n_trials = n_trials
        self.seed = seed
        self.study_timeout = study_timeout
        self.direction = direction
        self.study_name = study_name if study_name else "optuna_study"
        self.save_plots = save_plots

        # Set up logging
        self.logger = logging.getLogger()
        if not self.logger.handlers:
            self.logger.setLevel(logging.INFO)
            self.logger.addHandler(logging.FileHandler(f"{self.study_name}.log", mode="w"))
            optuna.logging.enable_propagation()
            optuna.logging.disable_default_handler()
        else:
            self.logger.handlers.clear()
            self.logger.setLevel(logging.INFO)
            self.logger.addHandler(logging.FileHandler(f"{self.study_name}.log", mode="w"))
            optuna.logging.enable_propagation()
            optuna.logging.disable_default_handler()

    def run_study(self):
        """
        Runs the optimization study.
        """
        self.logger.info("Start optimization.")

        self.study = optuna.create_study(
            direction=self.direction,
            sampler=optuna.samplers.TPESampler(seed=self.seed),
            pruner=self.pruner,
        )

        self.study.optimize(
            self.objective,
            n_trials=self.n_trials,
            timeout=self.study_timeout,
            show_progress_bar=True,
        )

        # Save study results to CSV
        df = self.study.trials_dataframe()
        df.to_csv(f"{self.study_name}_results.csv")
        # Save best parameters to CSV and pickle
        self.logger.info(f"Best params: {self.study.best_params}")
        self.save_best_params()

    def save_best_params(self):
        """
        Saves the best parameters to a CSV file.
        """
        best_params = pd.DataFrame(
            self.study.best_params.items(), columns=["parameter", "value"]
        )
        best_params.to_csv(f"{self.study_name}_best_params.csv", index=False)
        with open(f"{self.study_name}_best_params.pkl", "wb") as f:
            dump(self.study.best_params, f)
        self.logger.info("Best parameters saved in csv and pkl format.")

    def plot_results(self):
        """Generates and saves plotly plots of the study results."""

        # Plot optimization history
        optim_hist = optuna.visualization.plot_optimization_history(self.study)
        optim_hist.show()

        # Plot parameter importances
        p_import = optuna.visualization.plot_param_importances(self.study)
        p_import.show()

        # Plot parameter relationships
        contour = optuna.visualization.plot_contour(self.study)
        contour.show
        slice = optuna.visualization.plot_slice(self.study)
        slice.show()

        # Plot Timeline
        timeline = optuna.visualization.plot_timeline(self.study)
        timeline.show()

        # Parallel Coordinate Plot
        parallel = optuna.visualization.plot_parallel_coordinate(self.study)
        parallel.show()

        # Save plots
        if self.save_plots:
            optim_hist.write_image(f"{self.study_name}_optimization_history.png")
            p_import.write_image(f"{self.study_name}_param_importances.png")
            contour.write_image(f"{self.study_name}_contour.png")
            slice.write_image(f"{self.study_name}_slice.png")
            timeline.write_image(f"{self.study_name}_timeline.png")
            parallel.write_image(f"{self.study_name}_parallel.png")
            
            self.logger.info("Plots saved.")

## Elastic Net Example

In [8]:
# imports for lasso regression example
import numpy as np
from sklearn import datasets
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import optuna

In [10]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target


# Define the objective function for elastic net
def elastic_objective(trial):
    # Define cross-validation
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = []

    for step, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Standardize train and test sets
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Suggest hyperparameter values
        params = {
            "alpha": trial.suggest_float("alpha", 0.0, 1.0, log=False),
            "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
            "positive": trial.suggest_categorical("positive", [True, False]),
            "max_iter": trial.suggest_int("max_iter", 100, 10000, log=True),
            "fit_intercept": False,
            "tol": trial.suggest_float("tol", 1e-5, 1e-3, log=True),
        }

        # Create a Lasso regressor with the suggested alpha
        regressor = ElasticNet(**params, random_state=42)

        # Fit the model on scaled training data
        regressor.fit(X_train_scaled, y_train)

        # Predict and calculate MSE
        y_pred = regressor.predict(X_test_scaled)
        score = mean_squared_error(y_test, y_pred)

        scores.append(score)

        # Report intermediate score to Optuna
        trial.report(np.mean(score), step)

        # Check for pruning
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the average score over all folds
    return np.mean(scores)


# Create the pruner
pruner = optuna.pruners.MedianPruner(
    n_startup_trials=15, interval_steps=1
)

# Instantiate the tuner with the Lasso objective function
tuner = OptunaTuner(
    objective=elastic_objective,
    pruner=pruner,
    n_trials=100,
    seed=123,
    study_timeout=0.5 * 3600,
    direction="minimize",
    study_name="elastic_net_diabetes",
    save_plots=True,
)

# Run the study
tuner.run_study()

# Plot the results
tuner.plot_results()

  0%|          | 0/100 [00:00<?, ?it/s]

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.331e+05, tolerance: 3.757e+02

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.467e+05, tolerance: 3.741e+02

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.049e+

[I 2024-03-09 11:30:47,543] Trial 0 finished with value: 26192.19415362568 and parameters: {'alpha': 0.6964691855978616, 'l1_ratio': 0.28613933495037946, 'positive': False, 'max_iter': 2744, 'tol': 7.01799283113844e-05}. Best is trial 0 with value: 26192.19415362568.
[I 2024-03-09 11:30:47,550] Trial 1 finished with value: 26302.637741241222 and parameters: {'alpha': 0.9807641983846155, 'l1_ratio': 0.6848297385848633, 'positive': True, 'max_iter': 484, 'tol': 0.00028714378103928377}. Best is trial 0 with value: 26192.19415362568.
[I 2024-03-09 11:30:47,556] Trial 2 finished with value: 26153.098784606816 and parameters: {'alpha': 0.4385722446796244, 'l1_ratio': 0.05967789660956835, 'positive': False, 'max_iter': 231, 'tol': 2.2433834533274747e-05}. Best is trial 2 with value: 26153.098784606816.
[I 2024-03-09 11:30:47,563] Trial 3 finished with value: 26099.09717111968 and parameters: {'alpha': 0.5315513738418384, 'l1_ratio': 0.5318275870968661, 'positive': False, 'max_iter': 2808, 'to

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.915e+03, tolerance: 4.692e+02



[I 2024-03-09 11:30:47,748] Trial 22 finished with value: 26040.455928247775 and parameters: {'alpha': 0.1525563597193584, 'l1_ratio': 0.849097536018296, 'positive': False, 'max_iter': 541, 'tol': 2.585206981598495e-05}. Best is trial 22 with value: 26040.455928247775.
[I 2024-03-09 11:30:47,759] Trial 23 finished with value: 26046.028884515716 and parameters: {'alpha': 0.12955289287015526, 'l1_ratio': 0.7514780220575703, 'positive': False, 'max_iter': 609, 'tol': 2.8705423117628808e-05}. Best is trial 22 with value: 26040.455928247775.
[I 2024-03-09 11:30:47,773] Trial 24 finished with value: 26084.433068359183 and parameters: {'alpha': 0.0018730592122092604, 'l1_ratio': 0.8493483603573282, 'positive': False, 'max_iter': 619, 'tol': 5.495129838658247e-05}. Best is trial 22 with value: 26040.455928247775.
[I 2024-03-09 11:30:47,785] Trial 25 finished with value: 26043.099530576128 and parameters: {'alpha': 0.12215069795075262, 'l1_ratio': 0.9035193606967687, 'positive': False, 'max_ite

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.562e+02, tolerance: 2.228e+02

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.132e+05, tolerance: 9.549e+02

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.457e+

[I 2024-03-09 11:30:47,958] Trial 41 finished with value: 26037.579459370227 and parameters: {'alpha': 0.16612066100455697, 'l1_ratio': 0.9221441164439784, 'positive': False, 'max_iter': 234, 'tol': 2.2769349438977857e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-09 11:30:47,971] Trial 42 finished with value: 26046.464860623095 and parameters: {'alpha': 0.15299420353327295, 'l1_ratio': 0.9756292181507696, 'positive': False, 'max_iter': 286, 'tol': 2.301655034626176e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-09 11:30:47,982] Trial 43 pruned. 
[I 2024-03-09 11:30:47,993] Trial 44 pruned. 
[I 2024-03-09 11:30:48,006] Trial 45 finished with value: 26047.100932164918 and parameters: {'alpha': 0.05372851550525773, 'l1_ratio': 0.8334801682120019, 'positive': False, 'max_iter': 223, 'tol': 2.6093262790627616e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-09 11:30:48,016] Trial 46 pruned. 
[I 2024-03-09 11:30:48,026] Trial 47 pruned.

/tmp/ipykernel_190831/1114543482.py:104: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.

